In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.3/949.3 kB 26.5 MB/s eta 0:00:00


In [ ]:
import os
import cv2
import time
import torch
import pandas as pd
import numpy as np
from ultralytics import YOLO

# Load YOLOv8 Pose model
model = YOLO('yolo11n-pose.pt')

save_path = '/kaggle/working//output_pose_and_score.csv'
video_folder = '/kaggle/input/data-action/data_action/train'


# Get class names from folder structure
class_names = sorted(os.listdir(video_folder))

columns = ['video', 'frame', 'nose_x','nose_y','nose_s','left_eye_x','_left_eye_y','left_eye_s',
           'right_eye_x','right_eye_y','right_eye_s','left_ear_x','left_ear_y','left_ear_s',
           'right_ear_x','right_ear_y','right_ear_s','left_shoulder_x','left_shoulder_y',
           'left_shoulder_s','right_shoulder_x','right_shoulder_y','right_shoulder_s',
           'left_elbow_x','left_elbow_y','left_elbow_s','right_elbow_x','right_elbow_y','right_elbow_s',
           'left_wrist_x','left_wrist_y','left_wrist_s','right_wrist_x','right_wri  st_y','right_wrist_s',
           'left_hip_x','left_hip_y','left_hip_s','right_hip_x','right_hip_y','right_hip_s',
           'left_knee_x','left_knee_y','left_knee_s','right_knee_x','right_knee_y','right_knee_s',
           'left_ankle_x','left_ankle_y','left_ankle_s','right_ankle_x','right_ankle_y','right_ankle_s','label']

frame_size = [640, 640]

def normalize_points(points, width, height):
    points[..., 0] /= width
    points[..., 1] /= height
    return points

for label in class_names:
    label_folder = os.path.join(video_folder, label)
    videos = [f for f in os.listdir(label_folder) if f.endswith(('.mp4', '.avi'))]

    for vid in videos:
        print(f'Processing video: {vid} with label: {label}')
        df = pd.DataFrame(columns=columns)
        cap = cv2.VideoCapture(os.path.join(label_folder, vid))
        frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cls_idx = class_names.index(label)
        i = 1

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            frame = cv2.resize(frame, frame_size)

            results = model(frame)
            for result in results:
                if result.keypoints is not None and result.keypoints.xy is not None:
                    keypoints = result.keypoints.xy.cpu().numpy()
                    confs = result.keypoints.conf.cpu().numpy() if result.keypoints.conf is not None else np.full((keypoints.shape[0], keypoints.shape[1]), np.nan)
                    
                    if confs.ndim == 2:
                        confs = np.expand_dims(confs, axis=-1)

                    if keypoints.shape[0] > 0 and keypoints.shape[:2] == confs.shape[:2]:
                        keypoints = np.concatenate([keypoints, confs], axis=-1)
                        norm_keypoints = normalize_points(keypoints, frame_size[0], frame_size[1])
                        row = [vid, i] + norm_keypoints.flatten().tolist() + [cls_idx]
                    else:
                        row = [vid, i] + [np.nan] * (17 * 3) + [cls_idx]
                else:
                    row = [vid, i] + [np.nan] * (17 * 3) + [cls_idx]

                if len(row) == len(columns):
                    df.loc[len(df)] = row
                else:
                    print(f"Row length mismatch: expected {len(columns)}, got {len(row)}")

            i += 1

        cap.release()
        if os.path.exists(save_path):
            df.to_csv(save_path, mode='a', header=False, index=False)
        else:
            df.to_csv(save_path, mode='w', index=False)

print('Processing complete.')
